In [144]:
import sys, os
import pandas as pd
import geopandas as gpd
import skmob as skm

import h3

# as jupyter notebook cannot find __file__, import module and submodule path via current_folder
current_folder = globals()['_dh'][0]

PROJECT_SRC_PATH = os.path.realpath(os.path.join(current_folder, '..', '..', 'xml4uf'))
sys.path.append(PROJECT_SRC_PATH)

from ufo_map.Utils.helpers import *
from utils.utils import get_h3_points

In [156]:
df = pd.read_csv('/Users/Felix/Documents/Studium/PhD/05_Projects/03_Understand_Urban_Form_on_Mobility/03_Program/data/tripdata/waypoints/waypoints_100k.csv')
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat) ,crs=4326)

In [157]:
gdf.head()

,Unnamed: 0,Unnamed: 0.1,TripID,WaypointSequence,CaptureDate,lat,lon,SegmentID,ZoneName,Frc,DeviceID,RawSpeed,RawSpeedMetric,lon_proj,lat_proj,OSM_edge_1,OSM_edge_2,geometry
0,0,939,0864364a093e5c9d8b534bc3da3c1f58,0,2017-03-27T18:09:15.253Z,52.351000,13.099400,NaN,NaN,NaN,84bf44fb441a49031f0ab0ffa4de5f64,NaN,kph,370551.720553,5.801779e+06,232588936,232588923,POINT (13.09940 52.35100)
1,1,940,0864364a093e5c9d8b534bc3da3c1f58,1,2017-03-27T18:12:17.427Z,52.355463,13.117613,NaN,NaN,NaN,84bf44fb441a49031f0ab0ffa4de5f64,NaN,kph,371805.019294,5.802243e+06,241639517,241642425,POINT (13.11761 52.35546)
2,2,1092,0a95ff01129a456b19620b75e5329d6b,42,2017-03-27T16:02:37.000Z,52.325111,13.103566,NaN,NaN,NaN,264052c66423c0367645d09c35c91710,127.0,kph,370759.967762,5.798892e+06,275243986,290706780,POINT (13.10357 52.32511)
3,3,1093,0a95ff01129a456b19620b75e5329d6b,43,2017-03-27T16:03:07.000Z,52.331043,13.114602,NaN,NaN,NaN,264052c66423c0367645d09c35c91710,125.0,kph,371529.180570,5.799532e+06,31450590,268017004,POINT (13.11460 52.33104)
4,4,1094,0a95ff01129a456b19620b75e5329d6b,44,2017-03-27T16:03:37.000Z,52.334881,13.128057,NaN,NaN,NaN,264052c66423c0367645d09c35c91710,123.0,kph,372456.963044,5.799935e+06,6978263531,268017289,POINT (13.12806 52.33488)


In [239]:
# convert to h3 with hex8; 
# to do: adjust get_h3_points function to be more flexible with different crs

def discretize_space_time(gdf,
                            id_col = 'TripID',
                            datetime_col = 'CaptureDate',
                            hex_size=8,
                            time_step=30):
    """Function to convert waypoints to discrete space-time points on the hour
        in: 
        gdf:=gdf with one waypoint per row
        hex_size:= aperture_size
        time_step:=time in min
        out: gdf_out:= gdf[id,'t0','t1',...,'tn'], per t we allocate a position marked by hex_id or np.nan
    """
    print('...preparing spatial discretization for hex{}...'.format(hex_size))
    # create additonal column with hex_ids for each waypoint
    gdf_h3 = get_h3_points(gdf,'geometry',hex_size)
    
    # intialise output df
    gdf_out = pd.DataFrame(data=gdf_h3[id_col].unique(), columns=[id_col])

    print(gdf_out.head())

    print('...preparing temporal discretization...')
    # convert to datetime 
    gdf_h3[datetime_col] = pd.to_datetime(gdf_h3[datetime_col])
    
    # group data into x min time steps
    gdf_h3['time_floored']=gdf_h3.CaptureDate.dt.floor(str(time_step)+'min').dt.time

    for t, group in gdf_h3.groupby('time_floored'):
        print('discretizing for t = {}'.format(t))        
        
        # intialise discret time intervalls on trip id
        tx = 't_'+str(t.strftime('%H'))+str(t.strftime('%M'))
        gdf_out[tx] = np.nan
        
        # drop waypoints (duplictae case is when within time step, we have several waypoints) and take only first
        group.drop_duplicates(subset=[id_col], keep = False, inplace=True)
        
        # merge on id
        gdf_out = pd.merge(gdf_out,group[[id_col,'hex_id']], on=id_col,how='left')
        
        # add hex_id to t[x] where apliccable
        gdf_out[tx] = gdf_out.hex_id
        
        # drop hex_id col
        if 'hex_id' in gdf_out.columns: gdf_out = gdf_out.drop(columns='hex_id')

    # return gdf_out
    return gdf_out


In [241]:
discretize_space_time(gdf,id_col='TripID',datetime_col ='CaptureDate',hex_size=8,time_step=60)

...preparing spatial discretization for hex8...
                             TripID
0  0864364a093e5c9d8b534bc3da3c1f58
1  0a95ff01129a456b19620b75e5329d6b
2  113c27fdfe517cae7ea395f81f47e6ac
3  11960c47a1277e42a988396179b79b98
4  13b3987c07aa6b06b2b6c5e933f86bd9
...preparing temporal discretization...
discretizing for t = 00:00:00
discretizing for t = 01:00:00
discretizing for t = 02:00:00
discretizing for t = 03:00:00
discretizing for t = 04:00:00
discretizing for t = 05:00:00
discretizing for t = 06:00:00
discretizing for t = 07:00:00
discretizing for t = 08:00:00
discretizing for t = 09:00:00
discretizing for t = 10:00:00
discretizing for t = 11:00:00
discretizing for t = 12:00:00
discretizing for t = 13:00:00
discretizing for t = 14:00:00
discretizing for t = 15:00:00
discretizing for t = 16:00:00
discretizing for t = 17:00:00
discretizing for t = 18:00:00
discretizing for t = 19:00:00
discretizing for t = 20:00:00
discretizing for t = 21:00:00
discretizing for t = 22:00:00
discre

,TripID,t_0000,t_0100,t_0200,t_0300,t_0400,t_0500,t_0600,t_0700,t_0800,...,t_1400,t_1500,t_1600,t_1700,t_1800,t_1900,t_2000,t_2100,t_2200,t_2300
0,0864364a093e5c9d8b534bc3da3c1f58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0a95ff01129a456b19620b75e5329d6b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,113c27fdfe517cae7ea395f81f47e6ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,881f188ec3fffff,NaN,NaN,NaN,NaN,NaN,NaN
3,11960c47a1277e42a988396179b79b98,NaN,NaN,NaN,NaN,NaN,NaN,881f188ec7fffff,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13b3987c07aa6b06b2b6c5e933f86bd9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14714,d27088f42535033b48c791f933e58369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14715,f1c9bf9ba3ea1f599f3d087cdf6a054a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14716,0cb79cf5e77095318483f512a26096e9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14717,2ab7034a9bdc52e9155dd4f930d4263d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
gdf_h3

,Unnamed: 0,Unnamed: 0.1,TripID,WaypointSequence,CaptureDate,lat,lon,SegmentID,ZoneName,Frc,DeviceID,RawSpeed,RawSpeedMetric,lon_proj,lat_proj,OSM_edge_1,OSM_edge_2,geometry,lng,hex_id
0,0,939,0864364a093e5c9d8b534bc3da3c1f58,0,2017-03-27 18:09:15.253000+00:00,52.351000,13.099400,NaN,NaN,NaN,84bf44fb441a49031f0ab0ffa4de5f64,NaN,kph,370551.720553,5.801779e+06,232588936,232588923,POINT (13.09940 52.35100),13.099400,881f188523fffff
1,1,940,0864364a093e5c9d8b534bc3da3c1f58,1,2017-03-27 18:12:17.427000+00:00,52.355463,13.117613,NaN,NaN,NaN,84bf44fb441a49031f0ab0ffa4de5f64,NaN,kph,371805.019294,5.802243e+06,241639517,241642425,POINT (13.11761 52.35546),13.117613,881f188531fffff
2,2,1092,0a95ff01129a456b19620b75e5329d6b,42,2017-03-27 16:02:37+00:00,52.325111,13.103566,NaN,NaN,NaN,264052c66423c0367645d09c35c91710,127.0,kph,370759.967762,5.798892e+06,275243986,290706780,POINT (13.10357 52.32511),13.103566,881f188ec7fffff
3,3,1093,0a95ff01129a456b19620b75e5329d6b,43,2017-03-27 16:03:07+00:00,52.331043,13.114602,NaN,NaN,NaN,264052c66423c0367645d09c35c91710,125.0,kph,371529.180570,5.799532e+06,31450590,268017004,POINT (13.11460 52.33104),13.114602,881f188ec3fffff
4,4,1094,0a95ff01129a456b19620b75e5329d6b,44,2017-03-27 16:03:37+00:00,52.334881,13.128057,NaN,NaN,NaN,264052c66423c0367645d09c35c91710,123.0,kph,372456.963044,5.799935e+06,6978263531,268017289,POINT (13.12806 52.33488),13.128057,881f188eddfffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,14196915,2ab7034a9bdc52e9155dd4f930d4263d,14,2017-02-17 11:50:39+00:00,52.464570,13.129810,NaN,NaN,NaN,cda7cbbab5ee5fac3e40851821a2d08b,31.0,kph,372949.527918,5.814356e+06,278025029,278030290,POINT (13.12981 52.46457),13.129810,881f188647fffff
99996,99996,14196916,2ab7034a9bdc52e9155dd4f930d4263d,15,2017-02-17 11:50:54+00:00,52.465710,13.128900,NaN,NaN,NaN,cda7cbbab5ee5fac3e40851821a2d08b,40.0,kph,372890.997628,5.814484e+06,27324719,27324721,POINT (13.12890 52.46571),13.128900,881f188647fffff
99997,99997,14196917,2ab7034a9bdc52e9155dd4f930d4263d,16,2017-02-17 11:51:09+00:00,52.467250,13.127240,NaN,NaN,NaN,cda7cbbab5ee5fac3e40851821a2d08b,52.0,kph,372782.680502,5.814658e+06,27324721,27324719,POINT (13.12724 52.46725),13.127240,881f188647fffff
99998,99998,14196918,2ab7034a9bdc52e9155dd4f930d4263d,17,2017-02-17 11:51:24+00:00,52.468000,13.127000,NaN,NaN,NaN,cda7cbbab5ee5fac3e40851821a2d08b,33.0,kph,372768.541812,5.814742e+06,27324721,29846642,POINT (13.12700 52.46800),13.127000,881f188647fffff


In [178]:
gdf_h3['dates'] = pd.to_datetime(gdf_h3['CaptureDate']).dt.date
gdf_h3['time'] = pd.to_datetime(gdf_h3['CaptureDate']).dt.time


,Unnamed: 0,Unnamed: 0.1,TripID,WaypointSequence,CaptureDate,lat,lon,SegmentID,ZoneName,Frc,...,OSM_edge_2,geometry,lng,hex_id,datetime,start,elapsed,group,dates,time
0,0,939,0864364a093e5c9d8b534bc3da3c1f58,0,2017-03-27 18:09:15.253000+00:00,52.351000,13.099400,NaN,NaN,NaN,...,232588923,POINT (13.09940 52.35100),13.099400,881f188523fffff,2017-03-27 18:09:15.253000+00:00,2017-03-27 18:09:15.253000+00:00,0,1313,2017-03-27,18:09:15.253000
1,1,940,0864364a093e5c9d8b534bc3da3c1f58,1,2017-03-27 18:12:17.427000+00:00,52.355463,13.117613,NaN,NaN,NaN,...,241642425,POINT (13.11761 52.35546),13.117613,881f188531fffff,2017-03-27 18:12:17.427000+00:00,2017-03-27 18:12:17.427000+00:00,0,1314,2017-03-27,18:12:17.427000
2,2,1092,0a95ff01129a456b19620b75e5329d6b,42,2017-03-27 16:02:37+00:00,52.325111,13.103566,NaN,NaN,NaN,...,290706780,POINT (13.10357 52.32511),13.103566,881f188ec7fffff,2017-03-27 16:02:37+00:00,2017-03-27 16:02:37+00:00,0,1647,2017-03-27,16:02:37
3,3,1093,0a95ff01129a456b19620b75e5329d6b,43,2017-03-27 16:03:07+00:00,52.331043,13.114602,NaN,NaN,NaN,...,268017004,POINT (13.11460 52.33104),13.114602,881f188ec3fffff,2017-03-27 16:03:07+00:00,2017-03-27 16:03:07+00:00,0,1646,2017-03-27,16:03:07
4,4,1094,0a95ff01129a456b19620b75e5329d6b,44,2017-03-27 16:03:37+00:00,52.334881,13.128057,NaN,NaN,NaN,...,268017289,POINT (13.12806 52.33488),13.128057,881f188eddfffff,2017-03-27 16:03:37+00:00,2017-03-27 16:03:37+00:00,0,1648,2017-03-27,16:03:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,14196915,2ab7034a9bdc52e9155dd4f930d4263d,14,2017-02-17 11:50:39+00:00,52.464570,13.129810,NaN,NaN,NaN,...,278030290,POINT (13.12981 52.46457),13.129810,881f188647fffff,2017-02-17 11:50:39+00:00,2017-02-17 11:49:54+00:00,0,6998,2017-02-17,11:50:39
99996,99996,14196916,2ab7034a9bdc52e9155dd4f930d4263d,15,2017-02-17 11:50:54+00:00,52.465710,13.128900,NaN,NaN,NaN,...,27324721,POINT (13.12890 52.46571),13.128900,881f188647fffff,2017-02-17 11:50:54+00:00,2017-02-17 11:49:54+00:00,0,6998,2017-02-17,11:50:54
99997,99997,14196917,2ab7034a9bdc52e9155dd4f930d4263d,16,2017-02-17 11:51:09+00:00,52.467250,13.127240,NaN,NaN,NaN,...,27324719,POINT (13.12724 52.46725),13.127240,881f188647fffff,2017-02-17 11:51:09+00:00,2017-02-17 11:49:54+00:00,0,6998,2017-02-17,11:51:09
99998,99998,14196918,2ab7034a9bdc52e9155dd4f930d4263d,17,2017-02-17 11:51:24+00:00,52.468000,13.127000,NaN,NaN,NaN,...,29846642,POINT (13.12700 52.46800),13.127000,881f188647fffff,2017-02-17 11:51:24+00:00,2017-02-17 11:49:54+00:00,0,6998,2017-02-17,11:51:24


In [224]:
for t, group in gdf_h3.dt.floor('30min').dt.time.to_frame().groupby('CaptureDate'):
    print(group)
    break

AttributeError: 'GeoDataFrame' object has no attribute 'dt'

In [230]:
gdf_h3['time_floor']=gdf_h3.CaptureDate.dt.floor('30min').dt.time
for t, group in gdf_h3.groupby('time_floor'):
    print(t)
    break 

00:00:00


In [179]:
#gdf_h3['datetime'] = pd.to_datetime(df.CaptureDate)
#grouped_30min = df.groupby(pd.Grouper(key='CaptureDate', freq='30min'))
# find min time for each (location, TV) pair
gdf_h3['start'] = gdf_h3.groupby(['TripID', 'hex_id'])['time'].transform('min')


In [181]:
gdf_h3

,Unnamed: 0,Unnamed: 0.1,TripID,WaypointSequence,CaptureDate,lat,lon,SegmentID,ZoneName,Frc,...,OSM_edge_2,geometry,lng,hex_id,datetime,start,elapsed,group,dates,time
0,0,939,0864364a093e5c9d8b534bc3da3c1f58,0,2017-03-27 18:09:15.253000+00:00,52.351000,13.099400,NaN,NaN,NaN,...,232588923,POINT (13.09940 52.35100),13.099400,881f188523fffff,2017-03-27 18:09:15.253000+00:00,18:09:15.253000,0,1313,2017-03-27,18:09:15.253000
1,1,940,0864364a093e5c9d8b534bc3da3c1f58,1,2017-03-27 18:12:17.427000+00:00,52.355463,13.117613,NaN,NaN,NaN,...,241642425,POINT (13.11761 52.35546),13.117613,881f188531fffff,2017-03-27 18:12:17.427000+00:00,18:12:17.427000,0,1314,2017-03-27,18:12:17.427000
2,2,1092,0a95ff01129a456b19620b75e5329d6b,42,2017-03-27 16:02:37+00:00,52.325111,13.103566,NaN,NaN,NaN,...,290706780,POINT (13.10357 52.32511),13.103566,881f188ec7fffff,2017-03-27 16:02:37+00:00,16:02:37,0,1647,2017-03-27,16:02:37
3,3,1093,0a95ff01129a456b19620b75e5329d6b,43,2017-03-27 16:03:07+00:00,52.331043,13.114602,NaN,NaN,NaN,...,268017004,POINT (13.11460 52.33104),13.114602,881f188ec3fffff,2017-03-27 16:03:07+00:00,16:03:07,0,1646,2017-03-27,16:03:07
4,4,1094,0a95ff01129a456b19620b75e5329d6b,44,2017-03-27 16:03:37+00:00,52.334881,13.128057,NaN,NaN,NaN,...,268017289,POINT (13.12806 52.33488),13.128057,881f188eddfffff,2017-03-27 16:03:37+00:00,16:03:37,0,1648,2017-03-27,16:03:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,14196915,2ab7034a9bdc52e9155dd4f930d4263d,14,2017-02-17 11:50:39+00:00,52.464570,13.129810,NaN,NaN,NaN,...,278030290,POINT (13.12981 52.46457),13.129810,881f188647fffff,2017-02-17 11:50:39+00:00,11:49:54,0,6998,2017-02-17,11:50:39
99996,99996,14196916,2ab7034a9bdc52e9155dd4f930d4263d,15,2017-02-17 11:50:54+00:00,52.465710,13.128900,NaN,NaN,NaN,...,27324721,POINT (13.12890 52.46571),13.128900,881f188647fffff,2017-02-17 11:50:54+00:00,11:49:54,0,6998,2017-02-17,11:50:54
99997,99997,14196917,2ab7034a9bdc52e9155dd4f930d4263d,16,2017-02-17 11:51:09+00:00,52.467250,13.127240,NaN,NaN,NaN,...,27324719,POINT (13.12724 52.46725),13.127240,881f188647fffff,2017-02-17 11:51:09+00:00,11:49:54,0,6998,2017-02-17,11:51:09
99998,99998,14196918,2ab7034a9bdc52e9155dd4f930d4263d,17,2017-02-17 11:51:24+00:00,52.468000,13.127000,NaN,NaN,NaN,...,29846642,POINT (13.12700 52.46800),13.127000,881f188647fffff,2017-02-17 11:51:24+00:00,11:49:54,0,6998,2017-02-17,11:51:24


In [180]:

# subtract min time, then integer division in 30-minute intervals
gdf_h3['elapsed'] = (gdf_h3['time'] - gdf_h3['start']).dt.seconds // (30 * 60)

# assign group numbers
gdf_h3['group'] = gdf_h3.groupby(['TripID', 'hex_id', 'elapsed']).ngroup()

gdf_h3



TypeError: unsupported operand type(s) for -: 'datetime.time' and 'datetime.time'

In [ ]:
# if we have cleaned waypoint data with clear start and end points 
# --> this implies we have very fast all input data ready for RNN approach from Berke! 

In [152]:
# intialise output df
gdf_out = pd.DataFrame(data=gdf_h3.TripID.unique(), columns=['TripID'])

# group into hours
times = pd.DatetimeIndex(gdf_h3.CaptureDate)

#t_grouped = gdf_h3.groupby([times.hour, times.minute])
t_grouped = gdf_h3.groupby(times.hour)

# let's check
gdf_out


,TripID
0,0864364a093e5c9d8b534bc3da3c1f58
1,0a95ff01129a456b19620b75e5329d6b
2,113c27fdfe517cae7ea395f81f47e6ac
3,11960c47a1277e42a988396179b79b98
4,13b3987c07aa6b06b2b6c5e933f86bd9
...,...
14714,d27088f42535033b48c791f933e58369
14715,f1c9bf9ba3ea1f599f3d087cdf6a054a
14716,0cb79cf5e77095318483f512a26096e9
14717,2ab7034a9bdc52e9155dd4f930d4263d


In [153]:
for t, group in t_grouped:
    print('matching for t = {}'.format(t))        
    # intialise discret time intervalls on trip id
    gdf_out['t'+str(h)] = np.nan
    # drop waypoints (duplictae case is when within time step, we have several waypoints) and take only first
    group.drop_duplicates(subset='TripID', keep = False, inplace=True)
    # merge on TripID
    gdf_out = pd.merge(gdf_out,group[['TripID','hex_id']], on='TripID',how='left')
    # add hex_id to t[x] where apliccable
    gdf_out['t'+str(t)] = gdf_out.hex_id
    # drop hex_id col
    if 'hex_id' in gdf_out.columns: gdf_out = gdf_out.drop(columns='hex_id')
    
    

matching for t = 0
matching for t = 1
matching for t = 2
matching for t = 3
matching for t = 4
matching for t = 5
matching for t = 6
matching for t = 7
matching for t = 8
matching for t = 9
matching for t = 10
matching for t = 11
matching for t = 12
matching for t = 13
matching for t = 14
matching for t = 15
matching for t = 16
matching for t = 17
matching for t = 18
matching for t = 19
matching for t = 20
matching for t = 21
matching for t = 22
matching for t = 23


In [154]:
gdf_out

,TripID,t23,t0,t1,t2,t3,t4,t5,t6,t7,...,t13,t14,t15,t16,t17,t18,t19,t20,t21,t22
0,0864364a093e5c9d8b534bc3da3c1f58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0a95ff01129a456b19620b75e5329d6b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,113c27fdfe517cae7ea395f81f47e6ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,881f188ec3fffff,NaN,NaN,NaN,NaN,NaN
3,11960c47a1277e42a988396179b79b98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,881f188ec7fffff,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13b3987c07aa6b06b2b6c5e933f86bd9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14714,d27088f42535033b48c791f933e58369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14715,f1c9bf9ba3ea1f599f3d087cdf6a054a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14716,0cb79cf5e77095318483f512a26096e9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14717,2ab7034a9bdc52e9155dd4f930d4263d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
